In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow==2.8.0

     |████████████████████████████████| 462 kB 4.3 MB/s 


In [ ]:
!pip install keras_applications


     |████████████████████████████████| 50 kB 2.9 MB/s 


In [ ]:
import joblib
import pandas as pd
import glob
from google.colab.patches import cv2_imshow
import matplotlib
matplotlib.use("Agg")

import keras

import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation


from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow import keras
from tensorflow.keras.optimizers import schedules

# importing the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
#from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings

#from resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions


In [ ]:
from keras_applications.imagenet_utils import _obtain_input_shape
from keras_applications.imagenet_utils import get_submodules_from_kwargs
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import backend as k
from keras.layers import *
from keras.layers import ZeroPadding2D


In [ ]:
k.set_image_data_format('channels_last')

In [ ]:

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import _obtain_input_shape
#from keras.engine.topology import get_source_inputs
from tensorflow.keras.utils import get_source_inputs
from keras.utils.layer_utils import get_source_inputs
#from keras.utils import get_source_inputs
#from keras.utils.layer_utils import get_source_inputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load saved data

data =  joblib.load('/content/drive/MyDrive/graduation project/video/marwa/features/speech.joblib')

In [ ]:
data

,path,actor,file_name,emotion
0,/content/drive/MyDrive/graduation project/vide...,1,01-01-01-01-01-01-01.mp4,0
1,/content/drive/MyDrive/graduation project/vide...,1,01-01-01-01-02-01-01.mp4,0
2,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-02-01-01.mp4,1
3,/content/drive/MyDrive/graduation project/vide...,2,01-01-02-01-02-02-01.mp4,1
4,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-01-01-01.mp4,1
...,...,...,...,...
1051,/content/drive/MyDrive/graduation project/vide...,2,01-01-06-01-02-02-24.mp4,5
1052,/content/drive/MyDrive/graduation project/vide...,1,01-01-06-02-01-01-24.mp4,5
1053,/content/drive/MyDrive/graduation project/vide...,2,01-01-06-02-01-02-24.mp4,5
1054,/content/drive/MyDrive/graduation project/vide...,2,01-01-06-02-02-02-24.mp4,5


In [ ]:
train, valid_test = train_test_split(data, test_size=0.2, stratify=data['emotion'] , random_state=42) 
test, valid = train_test_split(valid_test, test_size=0.5, stratify=valid_test['emotion'] , random_state=42)

In [ ]:
len(data) , len(train) , len(test) , len(valid)

(1056, 844, 106, 106)

In [ ]:
train_videos =[]
train_labels=[]
c_train=0
s=0
for index, row in train.iterrows():
  s+=1
  print(s)
  pathh=row['path']
  for filename in glob.glob(pathh+'*.jpg'):
    if filename is not None:
      #print(filename)
      train_labels.append(row['emotion'])
      train_videos.append(cv2.imread(os.path.join(filename)))
      c_train +=1
x_train = np.array(train_videos)
y_train = np.array(train_labels)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
c_train , x_train.shape

(25320, (25320, 224, 224, 3))

In [ ]:
x_train

In [ ]:
print(K.image_data_format())  

channels_last


In [ ]:
test_videos =[]
test_labels=[]
#y_train=[]
c_test=0
for index, row in test.iterrows():
  print(index)
  pathh=row['path']
  for filename in glob.glob(pathh+'*.jpg'):
    if filename is not None:
      print(filename)
      test_labels.append(row['emotion'])
      test_videos.append(cv2.imread(os.path.join(filename)))
      c_test +=1
x_test = np.array(test_videos)
y_test = np.array(test_labels)

170
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_03/01-01-06-02-01-01-03.mp4/frames/frame_0.00.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_03/01-01-06-02-01-01-03.mp4/frames/frame_0.11.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_03/01-01-06-02-01-01-03.mp4/frames/frame_0.22.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_03/01-01-06-02-01-01-03.mp4/frames/frame_0.33.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_03/01-01-06-02-01-01-03.mp4/frames/frame_0.45.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_03/01-01-06-02-01-01-03.mp4/frames/frame_0.56.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_03/01-01-06-02-01-01-03.mp4/frames/frame_0.67.jpg
/content/drive/MyDrive/

In [ ]:
valid_videos =[]
valid_labels=[]
#y_train=[]
c_valid=0
s=0
for index, row in valid.iterrows():
  s+=1
  print(s)
  pathh=row['path']
  for filename in glob.glob(pathh+'*.jpg'):
    if filename is not None:
      #print(filename)
      valid_labels.append(row['emotion'])
      valid_videos.append(cv2.imread(os.path.join(filename)))
      c_valid +=1
x_valid = np.array(valid_videos)
y_valid = np.array(valid_labels)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106


In [ ]:
#rest50

   
"""Custom ResNet50 model for Keras.
# Reference:
- [Deep Residual Learning for Image Recognition](
    https://arxiv.org/abs/1512.03385) (CVPR 2016 Best Paper Award)
Adapted from code contributed by Rishav Giri.
"""


preprocess_input = imagenet_utils.preprocess_input

WEIGHTS_PATH = ('https://github.com/fchollet/deep-learning-models/'
                'releases/download/v0.2/'
                'resnet50_weights_tf_dim_ordering_tf_kernels.h5')
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.2/'
                       'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

backend = None
layers = None
models = None
keras_utils = None


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if k.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = tf.keras.layers.Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2, filters3 = filters
    if k.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)
    
    x = tf.keras.layers.Add()([x, shortcut])
    x = Activation('relu')(x)
    return x


def ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=1000,
             **kwargs):
    """Instantiates the ResNet50 architecture.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 32.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    global backend, layers, models, keras_utils
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=32,
                                      #data_format=backend.image_data_format(),
                                      data_format="channels_last",
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not k.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if k.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

######
    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        x = layers.Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D()(x)
        else:
            warnings.warn('The output shape of `ResNet50(include_top=False)` '
                          'has been changed since Keras 2.2.0.')

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = tf.keras.utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # Load weights.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file(
                'resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,
                cache_subdir='models',
                md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file(
                'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,
                cache_subdir='models',
                md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)
        if k.backend() == 'theano':
            keras_utils.convert_all_kernels_in_model(model)
    elif weights is not None:
        model.load_weights(weights)

    return model

In [ ]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7),strides=(2, 2), padding='same')(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(6, activation="softmax")(headModel)

# placing the head FC model on top of the base model 
model = Model(inputs=baseModel.input, outputs=headModel)

# looping over all layers in the base model and freeze them so that they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy',
		#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=opt,
	metrics=["accuracy"])



   16384/94653016 [..............................] - ETA: 13s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:233: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.


94666752/94653016 [==============================] - 1s 0us/step
[INFO] compiling model...


In [ ]:
# trained the head of the network for a few epochs (all other layers
# are frozen) -- this allows the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(x_train, y_train, batch_size = 32, validation_data=(x_valid, y_valid), epochs = 5)

"""
H = model.fit_generator(
	x_train, train_labels, 
	#batch_size=32,
	steps_per_epoch=len(x_train) // 32,
	validation_data=(x_valid, valid_labels),
	validation_steps=len(x_valid) // 32,
	epochs=20) 
	"""


[INFO] training head...
Epoch 1/5
792/792 [==============================] - 4705s 6s/step - loss: 1.2278 - accuracy: 0.5327 - val_loss: 0.9938 - val_accuracy: 0.6230
Epoch 2/5
792/792 [==============================] - 4470s 6s/step - loss: 0.8299 - accuracy: 0.7122 - val_loss: 0.8124 - val_accuracy: 0.6956
Epoch 3/5
792/792 [==============================] - 4580s 6s/step - loss: 0.6627 - accuracy: 0.7737 - val_loss: 0.7089 - val_accuracy: 0.7425
Epoch 4/5
792/792 [==============================] - 4512s 6s/step - loss: 0.5605 - accuracy: 0.8133 - val_loss: 0.6422 - val_accuracy: 0.7717
Epoch 5/5
792/792 [==============================] - 4583s 6s/step - loss: 0.4850 - accuracy: 0.8389 - val_loss: 0.5967 - val_accuracy: 0.7774


'\nH = model.fit_generator(\n\tx_train, train_labels, \n\t#batch_size=32,\n\tsteps_per_epoch=len(x_train) // 32,\n\tvalidation_data=(x_valid, valid_labels),\n\tvalidation_steps=len(x_valid) // 32,\n\tepochs=20) \n\t'

In [ ]:
# plotting the training loss and accuracy
N = 5 #epocs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig(args["plot"])

In [ ]:
# save model
print("[INFO] serializing network...")
model.save('/content/drive/My Drive/graduation project/video/marwa/paper_code_res/models/single_frame_speech_2')
print("MODEL SAVED")


[INFO] serializing network...
INFO:tensorflow:Assets written to: /content/drive/My Drive/graduation project/video/marwa/paper_code_res/models/single_frame_speech_2/assets
MODEL SAVED


In [ ]:
#save paths of test data
savedvX = joblib.dump(test, os.path.join('/content/drive/My Drive/graduation project/video/marwa/features/single_frame_speech_2/', 'test.joblib'))

In [ ]:
#load model
new_model=keras.models.load_model('/content/drive/My Drive/graduation project/video/marwa/paper_code_res/models/single_frame_speech_2')
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [ ]:
data =  joblib.load('/content/drive/MyDrive/graduation project/video/marwa/features/single_frame_speech_2/test.joblib')

In [ ]:
test_videos=[]
test_labels=[]
from pyparsing import results
def predict_video(video_path,emotion):
  Q = deque(maxlen=30)
  for filename in glob.glob(video_path+'*.jpg'):
    frame=cv2.imread(os.path.join(filename))
    test_videos.append(frame)
    test_labels.append(emotion)
    preds = new_model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
	  #Q.append(preds)

	# perform prediction averaging over the current history of
	# previous predictions
    results=np.array(Q).mean(axis=0)
	  #results = np.array(Q).mean(axis=0)
    i=np.argmax(results)
    labels=class_names[i]
	  #labels=class_names[i]
	  
  return i

In [ ]:
from collections import deque
class_names = ['neutral','calm', 'happy','sad','angry', 'fearful' ]
pred=[]
label=[]
for index, row in data.iterrows():
  pred.append(predict_video(row['path'],row['emotion']))
  label.append(row['emotion'])

In [ ]:
type(label)

list

In [ ]:
l=np.array(label)
p=np.array(pred)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import seaborn as sn
print(classification_report(label,pred))

acc = float(accuracy_score(label,pred))*100
print("----accuracy score %s ----" % acc)



              precision    recall  f1-score   support

           0       0.88      0.78      0.82         9
           1       0.86      0.90      0.88        20
           2       0.86      0.95      0.90        19
           3       1.00      1.00      1.00        19
           4       1.00      0.85      0.92        20
           5       0.95      1.00      0.97        19

    accuracy                           0.92       106
   macro avg       0.92      0.91      0.92       106
weighted avg       0.93      0.92      0.92       106

----accuracy score 92.45283018867924 ----


In [ ]:
l

array([2, 1, 2, 1, 1, 1, 3, 3, 3, 2, 4, 2, 1, 3, 4, 1, 3, 5, 1, 0, 0, 0,
       4, 5, 3, 5, 3, 2, 1, 4, 3, 4, 5, 3, 2, 0, 3, 2, 0, 1, 4, 5, 1, 4,
       3, 2, 2, 2, 2, 1, 3, 1, 1, 4, 5, 4, 1, 2, 2, 2, 2, 3, 0, 5, 4, 4,
       3, 5, 2, 4, 1, 5, 4, 5, 4, 1, 1, 5, 4, 1, 2, 1, 5, 4, 3, 4, 5, 4,
       2, 5, 2, 3, 5, 1, 0, 4, 3, 5, 5, 5, 3, 4, 3, 5, 0, 0])

In [ ]:
p

array([2, 1, 2, 1, 1, 1, 3, 3, 3, 2, 4, 2, 1, 3, 2, 1, 3, 5, 1, 0, 0, 0,
       4, 5, 3, 5, 3, 2, 1, 4, 3, 5, 5, 3, 2, 0, 3, 2, 0, 0, 4, 5, 1, 4,
       3, 2, 2, 2, 2, 1, 3, 1, 1, 4, 5, 4, 1, 2, 2, 1, 2, 3, 0, 5, 4, 4,
       3, 5, 2, 2, 1, 5, 4, 5, 4, 1, 2, 5, 4, 1, 2, 1, 5, 4, 3, 4, 5, 4,
       2, 5, 2, 3, 5, 1, 1, 4, 3, 5, 5, 5, 3, 4, 3, 5, 0, 1])

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(l,p)
print(cm)
#df_cm = pd.DataFrame(cm)
class_names = ['neutral','calm', 'happy','sad','angry', 'fearful' ]
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names,)
sn.heatmap(df_cm, annot=True, fmt='')
plt.show()

[[ 7  2  0  0  0  0]
 [ 1 18  1  0  0  0]
 [ 0  1 18  0  0  0]
 [ 0  0  0 19  0  0]
 [ 0  0  2  0 17  1]
 [ 0  0  0  0  0 19]]


In [ ]:
x_test = np.array(test_videos)
y_test = np.array(test_labels)

In [ ]:
# evaluating the network

print("[INFO] evaluating network...")
predictions = new_model.predict(x_test, batch_size=32)
print(classification_report(y_test,
	predictions.argmax(axis=1), target_names=class_names ))

[INFO] evaluating network...
              precision    recall  f1-score   support

     neutral       0.72      0.61      0.66       270
        calm       0.73      0.79      0.76       600
       happy       0.75      0.84      0.79       571
         sad       0.84      0.78      0.81       570
       angry       0.88      0.70      0.78       600
     fearful       0.78      0.90      0.83       570

    accuracy                           0.78      3181
   macro avg       0.78      0.77      0.77      3181
weighted avg       0.79      0.78      0.78      3181



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import seaborn as sn
cm = confusion_matrix(y_test,predictions.argmax(axis=1))
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names,)
sn.heatmap(df_cm, annot=True, fmt='')
plt.show()

In [ ]:
print(cm)

[[164  62   0  22  12  10]
 [ 42 471  76   1   0  10]
 [  6  62 477   6   9  11]
 [ 10  23  17 447  19  54]
 [  7  22  64  26 419  62]
 [  0   3   6  33  17 511]]


In [ ]:
loss, acc = new_model.evaluate(x_test, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

100/100 [==============================] - 6s 52ms/step - loss: 0.6651 - accuracy: 0.7825
Restored model, accuracy: 78.25%
